In [1]:
import pandas as pd
import numpy as np
import string
import spacy
import multiprocessing 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# nlp = spacy.load('en_core_web_lg')
nlp = spacy.load('en_vectors_web_lg')

OSError: [E050] Can't find model 'en_vectors_web_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
dataFile = "../data/cleaned.csv"
data1 = pd.read_csv(dataFile)
data2 = pd.read_csv('../data/hate_cleaned.csv')

In [ ]:
data2 = data2.drop(data2[data2['Class'] == 'neither'].index)
data2['Class'] = 1

In [ ]:
data[data.isnull().any(axis=1)]

In [ ]:
data[data['Class'] == 1].tail(n=10)

In [ ]:
ent_whitelist = set(['ADJ', 'ADV', 'PROPN', 'VERB', 'NOUN', 'INTJ', 'PRON', 'X', 'NUM'])
whitelist = set(string.ascii_lowercase + string.digits)

In [ ]:
megaSpam = pd.read_csv('../data/polluters.tsv', sep='\t')
megaNonSpam = pd.read_csv('../data/legit.tsv', sep='\t')

In [ ]:
megaSpam['Class'] = 1
megaNonSpam['Class'] = 0

In [ ]:
data = pd.concat([data1, data2, megaSpam, megaNonSpam])
data.head()

In [ ]:
data.tail()

In [ ]:
def cleanWord(word):
    word = "".join([c for c in word.lower() if c in whitelist])
    return word

def invalidWord(word):
    if (len(word) == 0 or word[0] == '@' or word == '-pron-'):
        return True
    word = cleanWord(word)
    return len(word) == 0

def cleanText(text):
    if not isinstance(text, str):
        print('Found error: {}, {}'.format(text, type(text)))
        text = str(text)
#     output = []
#     doc = nlp(text)
#     for tok in doc:
#         if tok.pos_ in ent_whitelist:
#             output.append(tok.lemma_)
    words = text.split(' ')
    output = [cleanWord(word) for word in words if not invalidWord(word)]
    return ' '.join(output)

with multiprocessing.Pool(8) as p:
#     cleanedData = p.map(cleanText, cleanData, 100)
    cleanData = p.map(cleanText, data['Text'], 100)
#     for text in data["Text"]:
#     #     print(type(text))
#         cleanData.append(cleanText(str(text)))

In [ ]:
cleanData = [' '.join([x.lower() for x in y.split(' ') if len(x) > 0 and x[0] != '@']) for y in cleanData]

In [ ]:
cleanData[-12312]

In [ ]:
# vectorizer = CountVectorizer()
# for i in range(0, len(cleanData) - 9999, 10000):
# #     X = vectorizer.fit_transform(cleanData[i:i+10000])
    

In [ ]:


X.shape
# X.toarray().shape

In [ ]:
# encoder = LabelEncoder()
# y = encoder.fit_transform(data["Class"])
y = data['Class']

In [ ]:
print(y.shape)
print(len(cleanData))

In [ ]:
clean_train, clean_test, y_train, y_test = train_test_split(cleanData, y, test_size=0.25, random_state=42)
print(len(clean_test), len(y_test))
print(len(clean_train), len(y_train))
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(clean_train)

# classifier = GradientBoostingClassifier()
classifier = LogisticRegression()
classifier.fit(X_vec, y_train)

y_vec = vectorizer.transform(clean_test)
y_hat = classifier.predict(y_vec)


In [ ]:
classifier = GradientBoostingClassifier()
classifier.fit(X, y)

In [ ]:
from collections import Counter
Counter(y)
# Counter(y)

In [ ]:
accuracy = accuracy_score(y_test, y_hat)
accuracy

In [ ]:
accuracy = accuracy_score(y_test, y_hat)
accuracy

# PICKLE EVERYTHING

In [ ]:
import pickle
with open('thepickle_LR_beta.pkl', 'bw') as f:
#     pickle.dump([encoder, vectorizer, classifier], f)
    pickle.dump([vectorizer, classifier], f)

In [ ]:
coefs = classifier.coef_[0]

In [ ]:
k = 100
top_k = np.argpartition(coefs, -k)[-k:]
top_k_sorted=top_k[np.argsort(coefs[top_k])]

In [ ]:
feats = vectorizer.get_feature_names()
[feats[x] for x in top_k_sorted]